In [1]:
from aiida import orm
import aiida

aiida.load_profile('quicksetup')

Profile<uuid='152ba6b1f4ad4726905c46d707406b30' name='quicksetup'>

In [2]:
from aiida import engine
from aiida import orm
import os

from aiida_opsp.calcjob import OncvPseudoCalculation

computer = orm.load_computer('localhost')
code = orm.Code(label='oncv4-0', remote_computer_exec=[computer, '/home/jyu/Projects/WP-OPSP/bin/oncvpsp.x'], input_plugin_name='opsp.pseudo.oncv')


def get_inputs(rc_s, qcut_s):
    conf_name = orm.Str('Li-s')
    angular_momentum_settings = orm.Dict(
        dict={
            's': {
                'rc': rc_s,
                'ncon': 4,
                'nbas': 8,
                'qcut': qcut_s,
                'nproj': 2,
                'debl': 1.0,
            },
            'p': {
                'rc': 1.1,
                'ncon': 4,
                'nbas': 8,
                'qcut': 9.0,
                'nproj': 2,
                'debl': 1.0,
            }, 
        }
    )
    local_potential_settings = orm.Dict(
        dict={
            'llcol': 4, # fix
            'lpopt': 5, # 1-5, algorithm enum set
            'rc(5)': 1.1,
            'dvloc0': 0.0,
        }
    )
    nlcc_settings = orm.Dict(
        dict={
            'icmod': 0,
            'fcfact': 0.25,
        }
    )
    inputs = {
        'code': code,
        'conf_name': conf_name,
        'lmax': orm.Int(1),
        'angular_momentum_settings': angular_momentum_settings,
        'local_potential_settings': local_potential_settings,
        'nlcc_settings': nlcc_settings,
        'run_atomic_test': orm.Bool(True),
        'dump_psp': orm.Bool(False),
        'metadata': {
            'options': {
                'resources': {
                    'num_machines': int(1)
                },
                'max_wallclock_seconds': int(60),
                'withmpi': False,
            },
            # 'dry_run':True,
        }
    }
    
    return inputs

In [3]:
import numpy as np
import time
from aiida import orm

group = orm.Group(label='li_s_rc_qcut_20x20_0511')
group.store()
# group = orm.load_group(label='li_s_rc_qcut_20x20_0511')

lst = [(rc, qcut) for rc in np.linspace(1.1, 2.0, 19) for qcut in np.linspace(2, 12, 21)]

for rc, qcut in lst[:]:
    rc = round(rc, 2)
    qcut = round(qcut, 2)
    
    inputs = get_inputs(rc, qcut)
    node = engine.submit(OncvPseudoCalculation, **inputs)
    node.description = f'rc_s={rc}, qcut_s={qcut}'
    # print(f'rc_s={rc}, qcut_s={qcut}')
    group.add_nodes(node)
    
    time.sleep(1)

In [ ]:
# import numpy as np
# import time
# from aiida import orm

# # group = orm.Group(label='li_s_rc_qcut-zoom-in')
# # group.store()
# group = orm.load_group(label='li_s_rc_qcut-zoom-in')

# lst = [(rc, qcut) for rc in np.linspace(1.4, 1.4, 1) for qcut in np.linspace(8, 9, 11)]

# for rc, qcut in lst[:]:
#     rc = round(rc, 2)
#     qcut = round(qcut, 2)
    
#     inputs = get_inputs(rc, qcut)
#     node = engine.submit(OncvPseudoCalculation, **inputs)
#     node.description = f'rc_s={rc}, qcut_s={qcut}'
#     # print(f'rc_s={rc}, qcut_s={qcut}')
#     group.add_nodes(node)
    
#     time.sleep(1)